# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
import winsound
from tqdm import tqdm
tqdm.pandas()

In [3]:
def get_tick_time(data):
    data['df']['sno'] = data['df'].index
    data['df']['tick']      = (data["df"]['Ask'] + data["df"]['Bid'])/2
    data['df']['DateTime_frmt']   = [dt.datetime.strptime(x.split(".")[0],"%Y%m%d %H:%M:%S") for x in data["df"]['DateTime']]
    data['df'] = data['df'][['sno', 'DateTime_frmt', 'tick']]
    return(data)

In [4]:
def get_ohlc(data):
    data['df_bars'] = pd.DataFrame()
    data['df_bars']['sno'] = data['df'].resample(data['candle_size'], on='DateTime_frmt').sno.first()
    data['df_bars']['ts'] = data['df'].resample(data['candle_size'], on='DateTime_frmt').tick.min().index
    data['df_bars']['o'] = data['df'].resample(data['candle_size'], on='DateTime_frmt').tick.first().values
    data['df_bars']['h'] = data['df'].resample(data['candle_size'], on='DateTime_frmt').tick.max().values
    data['df_bars']['l'] = data['df'].resample(data['candle_size'], on='DateTime_frmt').tick.min().values
    data['df_bars']['c'] = data['df'].resample(data['candle_size'], on='DateTime_frmt').tick.last().values
    data['df_bars'] = data['df_bars'].reset_index(drop=True)
    return(data)

In [10]:
%%time
data['df'] = pd.read_csv('data/EURUSD_2021.csv', nrows=1000)

Wall time: 8.07 ms


In [11]:
data['candle_size'] = '1T'

In [12]:
data = get_tick_time(data)
data = get_ohlc(data)

In [13]:
data['df_bars']

,sno,ts,o,h,l,c
0,0.0,2021-01-03 22:00:00,1.224035,1.224035,1.223835,1.224005
1,28.0,2021-01-03 22:01:00,1.223915,1.224260,1.223905,1.224055
2,43.0,2021-01-03 22:02:00,1.224060,1.224060,1.223925,1.223925
3,55.0,2021-01-03 22:03:00,1.223935,1.224005,1.223815,1.223830
4,69.0,2021-01-03 22:04:00,1.223840,1.223875,1.223160,1.223535
...,...,...,...,...,...,...
56,877.0,2021-01-03 22:56:00,1.223485,1.223495,1.223350,1.223350
57,897.0,2021-01-03 22:57:00,1.223355,1.223415,1.223355,1.223390
58,907.0,2021-01-03 22:58:00,1.223405,1.223420,1.223385,1.223385
59,914.0,2021-01-03 22:59:00,1.223390,1.223435,1.223300,1.223435


In [15]:
data['df']

,sno,DateTime_frmt,tick
0,0,2021-01-03 22:00:00,1.224035
1,1,2021-01-03 22:00:00,1.224025
2,2,2021-01-03 22:00:00,1.224020
3,3,2021-01-03 22:00:00,1.224015
4,4,2021-01-03 22:00:00,1.224005
...,...,...,...
995,995,2021-01-03 23:00:13,1.224060
996,996,2021-01-03 23:00:13,1.224075
997,997,2021-01-03 23:00:14,1.224065
998,998,2021-01-03 23:00:14,1.224040
